# Ejercicios Semana 4

1. Agrega Gemini a la mezcla de código cerrado.
2. Prueba más modelos de código abierto como CodeLlama y StarCoder, y mira si puedes lograr que CodeGemma funcione.
3. Crea una herramienta de código que agrega automáticamente docstrings y comentarios.
4. Crea una herramienta de generación de código que escribe casos de pruebas unitarias.
5. Crea un generador de código que escribe código comercial para comprar y vender acciones en un entorno simulado, basado en una API determinada.

In [1]:
# imports

import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [32]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

# Indicar si la arquitectura del dispositivo es ARM
ARM_ARCH=False

In [3]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"

In [4]:
system_message = "Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. "
system_message += "Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
system_message += "La respuesta C++ debe producir una salida idéntica en el menor tiempo posible."

In [ ]:
def user_prompt_for(python):
    user_prompt = "Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible."
    user_prompt += "Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios."
    user_prompt += "Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos."
    user_prompt += "Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. "
    user_prompt += "Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes de C++ necesarios, como iomanip.\n\n"    
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [9]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [10]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [11]:
exec(pi)

Result: 3.141592658589
Execution Time: 15.042635 seconds


In [12]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <chrono>
#include <iomanip>

double calculate(long iterations, int param1, int param2) {
    double result = 1.0;
    for (long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::steady_clock::now();
    double result = calculate(100000000L, 4, 1) * 4;
    auto end_time = std::chrono::steady_clock::now();

    std::chrono::duration<double> execution_time = end_time - start_time;

    // Output with 12 decimal places as in Python's formatted print
    std::cout << "Result: " << std::fixed << std::setprecision(12) << result << std::endl;
    std::cout << "Execution Time: " << std::fixed << std::setprecision(6) << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [13]:
exec(pi)

Result: 3.141592658589
Execution Time: 15.080043 seconds


In [33]:
# Arquitectura ARM
if ARM_ARCH:
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.001195 seconds


In [15]:
optimize_claude(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        long long j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100000000LL, 4, 1) * 4.0;
    auto end_time = std::chrono::high_resolution_clock::now();
    
    std::chrono::duration<double> elapsed = end_time - start_time;
    
    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << std::setprecision(6);
    std::cout << "Execution Time: " << elapsed.count() << " seconds" << std::endl;
    
    return 0;
}
```

In [34]:
if ARM_ARCH:
    # Arquitectura ARM
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.001036 seconds


In [17]:
python_hard = """
def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [18]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 72.441455 seconds


In [19]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <iomanip>
#include <vector>
#include <chrono>
#include <limits>

// Linear Congruential Generator
class LCG {
public:
    LCG(unsigned int seed, unsigned int a=1664525, unsigned int c=1013904223, unsigned int m=1U << 32)
        : value(seed), a(a), c(c), m(m) {}

    unsigned int next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    unsigned int value;
    unsigned int a;
    unsigned int c;
    unsigned int m;
};

// Calculate the maximum subarray sum
long long max_subarray_sum(int n, unsigned int seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    long long max_sum = std::numeric_limits<long long>::min();
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];

In [35]:
if ARM_ARCH:
    # Arquitectura ARM
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.001064 seconds


In [21]:
optimize_claude(python_hard)

```cpp
#include <iostream>
#include <iomanip>
#include <vector>
#include <chrono>
#include <cstdint>
#include <limits>

class LCG {
private:
    uint64_t value;
    static const uint64_t a = 1664525;
    static const uint64_t c = 1013904223;
    static const uint64_t m = 4294967296ULL; // 2^32
    
public:
    LCG(uint64_t seed) : value(seed) {}
    
    uint64_t next() {
        value = (a * value + c) % m;
        return value;
    }
};

long long max_subarray_sum(int n, uint64_t seed, int min_val, int max_val) {
    LCG lcg_gen(seed);
    std::vector<int> random_numbers(n);
    
    int range = max_val - min_val + 1;
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = static_cast<int>(lcg_gen.next() % range) + min_val;
    }
    
    long long max_sum = std::numeric_limits<long long>::min();
    
    // Kadane's algorithm - O(n) instead of O(n^2)
    long long current_sum = 0;
    bool has_positive = false;
    
    for (int i = 0; i < n; ++i) {
        current_sum += rand

In [36]:
if ARM_ARCH:
    # Arquitectura ARM
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.001128 seconds


In [23]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [24]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [25]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Modelo Desconocido")
    for stream_so_far in result:
        yield stream_so_far

In [26]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Código en Python:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
        convert = gr.Button("Convertir código")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [27]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [28]:
def execute_cpp(code):
        write_output(code)
        try:
            if ARM_ARCH:
                # Arquitectura ARM
                compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m2", "-mcpu=apple-m2", "-o", "optimized", "optimized.cpp"]
            else:
                # Arquitectura Intel
                compile_cmd = ["clang++", "-O3", "-std=c++17", "-march=x86-64", "-o", "optimized", "optimized.cpp"]

            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [29]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [30]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Código en Python:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="Código en C++:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

/tmp/ipykernel_6982/1753851839.py:1: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=css) as ui:


Opening in existing browser session.
* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [38]:
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer
from dotenv import load_dotenv, find_dotenv
import os

PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [39]:
loaded = load_dotenv(find_dotenv())

if loaded:
    print("OK: .env file loaded sucessfully")
else:
    print("ERROR: .env file not loaded")

hf_token = os.getenv('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

OK: .env file loaded sucessfully


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
code_qwen = "Qwen/CodeQwen1.5-7B-Chat"
code_gemma = "google/codegemma-7b-it"
CODE_QWEN_URL = "https://mo1u84h30mt5vns6.us-east-1.aws.endpoints.huggingface.cloud"
CODE_GEMMA_URL = "https://c5hggiyqachmgnqg.us-east-1.aws.endpoints.huggingface.cloud"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(code_qwen)
messages = messages_for(pi)
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
print(text)

In [ ]:
client = InferenceClient(CODE_QWEN_URL, token=hf_token)
stream = client.text_generation(text, stream=True, details=True, max_new_tokens=1000)
for r in stream:
    print(r.token.text, end = "")

In [ ]:
def stream_code_qwen(python):
    tokenizer = AutoTokenizer.from_pretrained(code_qwen)
    messages = messages_for(python)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    client = InferenceClient(CODE_QWEN_URL, token=hf_token)
    stream = client.text_generation(text, stream=True, details=True, max_new_tokens=700)
    result = ""
    for r in stream:
        result += r.token.text
        yield result    

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="CodeQwen":
        result = stream_code_qwen(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "CodeQwen"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

# 1. Agrega Gemini a la mezcla de código cerrado

In [ ]:
import os
from google import genai
from dotenv import load_dotenv
from google.genai import types

load_dotenv()
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY', 'your-key-if-not-using-env')
gemini = genai.Client()
GEMINI_MODEL = "gemini-2.5-flash-lite"

In [ ]:
def extract_code(text):
    # Si el modelo devolvió formato markdown de código
    if "```" in text:
        # Busca el inicio del bloque cpp (o genérico)
        if "```cpp" in text:
            text = text.split("```cpp")[1]
        else:
            text = text.split("```")[1]

        # Corta hasta el cierre del bloque
        text = text.split("```")[0]

    return text.strip()

def optimize_gemini(python):
    prompt = system_message + "\n\n" + user_prompt_for(python)
    response = gemini.models.generate_content(
        model=GEMINI_MODEL,
        contents=user_prompt_for(python),
        config=types.GenerateContentConfig(
            system_instruction=system_message,
            max_output_tokens=2000,
            temperature=0.0,
            top_p=0.95
        ),
    )

    cpp_code = extract_code(response.text)
    print(cpp_code)

    write_output(cpp_code)

In [ ]:
optimize_gemini(pi)

In [ ]:
if ARM_ARCH:
    # Arquitectura ARM
    !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
else:
    # Arquitectura Intel
    !clang++ -O3 -std=c++17 -march=native -o optimized optimized.cpp

!./optimized

In [ ]:
def stream_gemini(python):
    prompt = system_message + "\n\n" + user_prompt_for(python)
    response = gemini.models.generate_content_stream(
        model=GEMINI_MODEL,
        contents=user_prompt_for(python),
        config=types.GenerateContentConfig(
            system_instruction=system_message,
            max_output_tokens=2000,
            temperature=0.0,
            top_p=0.95
        ),
    )
    reply = ""
    for chunk in response:
        fragment = chunk.text or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    else:
        raise ValueError("Modelo Desconocido")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

# 2. Prueba más modelos de código abierto

In [68]:
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer
import transformers
import torch

code_llama = "codellama/CodeLlama-7b-Instruct-hf"
CODE_LLAMA_URL = "https://ta3za4j1cdqekxq4.us-east-1.aws.endpoints.huggingface.cloud"

In [69]:
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [70]:
# Test rápido del endpoint
client = InferenceClient(CODE_LLAMA_URL, token=hf_token)
test = client.text_generation("print hello world in C++", max_new_tokens=50)
print(test)

print hello world in C++

```cpp
#include <iostream>

int main() {
    std::cout << "Hello, World!" << std::endl;
    return 0;
}
```

print hello world in C



In [71]:
tokenizer = AutoTokenizer.from_pretrained(code_llama)
messages = messages_for(pi)
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [72]:
print(text)

<s>[INST] <<SYS>>
Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. La respuesta C++ debe producir una salida idéntica en el menor tiempo posible.
<</SYS>>

Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible.Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios.Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos.Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes 

In [74]:
client = InferenceClient(CODE_LLAMA_URL, token=hf_token)
response = client.text_generation(
    text,
    stream=False,
    details=True,
    max_new_tokens=1000
)

print(response, end = "")

/home/isaac/workspace/llm_engineering/.venv/lib/python3.13/site-packages/huggingface_hub/inference/_client.py:2367: UserWarning: API endpoint/model for text-generation is not served via TGI. Ignoring following parameters: details.
  warnings.warn(
/home/isaac/workspace/llm_engineering/.venv/lib/python3.13/site-packages/huggingface_hub/inference/_client.py:2373: UserWarning: API endpoint/model for text-generation is not served via TGI. Parameter `details=True` will be ignored meaning only the generated text will be returned.
  warnings.warn(


<s>[INST] <<SYS>>
Eres un asistente que reimplementa código Python en C++ de alto rendimiento para una Mac M2. Responde solo con código C++; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. La respuesta C++ debe producir una salida idéntica en el menor tiempo posible.
<</SYS>>

Reescribe este código Python en C++ con la implementación más rápida posible que produzca una salida idéntica en el menor tiempo posible.Responde solo con código C++; no expliques tu trabajo más allá de algunos comentarios.Manten la implementación de la generación de números aleatorios idénticos para que los resultados de la coincidencia sean exactos.Responde solo con código C++; no añadas nada más que código; usa los comentarios con moderación y no proporciones ninguna explicación más allá de comentarios ocasionales. Presta atención a los tipos de números para asegurar que no haya desbordamientos de int (overflow). Recuerda incluir todos los paquetes 

In [100]:

def stream_code_llama(python):
    tokenizer = AutoTokenizer.from_pretrained(code_llama)
    messages = messages_for(python)

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    client = InferenceClient(CODE_LLAMA_URL, token=hf_token)
    response = client.text_generation(text, stream=False, max_new_tokens=700)
    result = str(response)

    if '[/INST]' in result:
        result = result.split('[/INST]')[-1].strip()

    result = result.replace('```cpp', '').replace('```', '').strip()

    # Reformatear
    result = result.replace('; ', ';\n')
    result = result.replace('{ ', '{\n')
    result = result.replace(' }', '\n}')
    result = result.replace('} ', '}\n')
    result = result.replace('#include', '\n#include')

    while '\n\n\n' in result:
        result = result.replace('\n\n\n', '\n\n')

    # Usar yield UNA SOLA VEZ con el resultado completo
    yield result.strip()

In [101]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_gemini(python)
    elif model=="CodeLlama":
        result = stream_code_llama(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [102]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convierte código de Python a C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini", "CodeLlama"], label="Selecciona el modelo", value="GPT")
    with gr.Row():
        convert = gr.Button("Convertir el código")
    with gr.Row():
        python_run = gr.Button("Ejecutar Python")
        cpp_run = gr.Button("Ejecutar C++")
    with gr.Row():
        python_out = gr.TextArea(label="Resultado en Python:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Resultado en C++:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

/tmp/ipykernel_6982/2423202974.py:1: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=css) as ui:


El código generado por CodeLlama contiene errores y no se puede compilar:

```
An error occurred:
optimized.cpp:14:66: error: indirection requires pointer operand ('int' invalid)
int lcg(int seed, int a = 1664525, int c = 1013904223, int m = 2**32) {
                                                                 ^~~
optimized.cpp:18:9: error: unknown type name 'yield'
        yield value;
        ^
optimized.cpp:25:5: error: use of undeclared identifier 'lcg_gen'
    lcg_gen = lcg(seed);
    ^
optimized.cpp:26:5: error: use of undeclared identifier 'random_numbers'
    random_numbers = new int[n];
    ^
optimized.cpp:30:9: error: use of undeclared identifier 'random_numbers'
        random_numbers[i] = (next(lcg_gen) % (max_val - min_val + 1) + min_val);
        ^
optimized.cpp:30:35: error: use of undeclared identifier 'lcg_gen'
        random_numbers[i] = (next(lcg_gen) % (max_val - min_val + 1) + min_val);
                                  ^
optimized.cpp:33:19: error: use of undeclared identifier 'INT_MIN'
    int max_sum = INT_MIN;
                  ^
optimized.cpp:41:28: error: use of undeclared identifier 'random_numbers'
            current_sum += random_numbers[j];
                           ^
optimized.cpp:56:5: error: use of undeclared identifier 'lcg_gen'
    lcg_gen = lcg(initial_seed);
    ^
optimized.cpp:60:25: error: use of undeclared identifier 'lcg_gen'
        int seed = next(lcg_gen);
                        ^
optimized.cpp:86:14: error: expected '}'
    return 0;
             ^
optimized.cpp:68:12: note: to match this '{'
int main() {
           ^
11 errors generated.
```

# 3. Crea una herramienta de código que agrega automáticamente docstrings y comentarios

In [103]:
# System message para agregar docstrings y comentarios
system_message = "Eres un asistente experto en documentación de código. "
system_message += "Tu tarea es agregar docstrings detallados y comentarios útiles al código que te proporcionen. "
system_message += "Sigue las mejores prácticas de documentación del lenguaje (PEP 257 para Python, JSDoc para JavaScript, etc.). "
system_message += "Responde SOLO con el código completo con los docstrings y comentarios agregados, sin explicaciones adicionales."

In [104]:
def docstring_prompt_for(code, language="Python"):
    user_prompt = f"Agrega docstrings completos y comentarios útiles al siguiente código {language}. "
    user_prompt += "Los docstrings deben incluir: descripción de la función/clase, parámetros, tipo de retorno y ejemplos cuando sea apropiado. "
    user_prompt += "Los comentarios deben explicar la lógica compleja pero sin ser redundantes. "
    user_prompt += "Responde SOLO con el código mejorado, sin explicaciones adicionales.\n\n"
    user_prompt += code
    return user_prompt

def messages_for_docstring(code, language="Python"):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": docstring_prompt_for(code, language)}
    ]

In [105]:
def add_docstrings_gpt(code, language="Python"):
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for_docstring(code, language),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','').strip()

def add_docstrings_claude(code, language="Python"):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=docstring_system_message,
        messages=[{"role": "user", "content": docstring_prompt_for(code, language)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```python\n','').replace('```','').strip()

def add_docstrings(code, model, language="Python"):
    if model == "GPT":
        result = add_docstrings_gpt(code, language)
    elif model == "Claude":
        result = add_docstrings_claude(code, language)
    else:
        raise ValueError("Modelo desconocido")

    for stream_so_far in result:
        yield stream_so_far

In [ ]:
# Código de ejemplo sin docstrings
code_example = """
def calculate_fibonacci(n):
    if n <= 1:
        return n
    return calculate_fibonacci(n-1) + calculate_fibonacci(n-2)

def process_data(items, threshold):
    result = []
    for item in items:
        if item > threshold:
            result.append(item * 2)
    return result
"""

with gr.Blocks(css=css) as ui:
    gr.Markdown("##Agregar Docstrings y Comentarios")

    with gr.Row():
        with gr.Column():
            code_input = gr.Code(label="Código Original:", language="python", lines=15, value=code_example)
        with gr.Column():
            code_output = gr.Code(label="Código con Docstrings:", language="python", lines=15)

    with gr.Row():
        language = gr.Dropdown(
            ["Python", "JavaScript", "Java", "C++"],
            label="Lenguaje",
            value="Python"
        )
        model = gr.Dropdown(
            ["GPT", "Claude"],
            label="Modelo",
            value="GPT"
        )

    with gr.Row():
        convert_btn = gr.Button("Agregar Docstrings", variant="primary")

    convert_btn.click(
        add_docstrings,
        inputs=[code_input, model, language],
        outputs=[code_output]
    )

ui.launch(inbrowser=True)

/tmp/ipykernel_6982/854007347.py:16: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=css) as ui:


In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Código Origen:", lines=10, value=pi)
        cpp = gr.Textbox(label="Código con Dostrings y Comentarios:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Selecciona el modelo", value="GPT")
        convert = gr.Button("Convertir código")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

# 4. Crea una herramienta de generación de código que escribe casos de pruebas unitarias

In [115]:
system_message = "Eres un asistente experto en testing de software. "
system_message += "Analiza el código proporcionado, detecta el lenguaje y genera pruebas unitarias completas usando el framework más apropiado. "
system_message += "Incluye casos normales, edge cases y casos de error. "
system_message += "Responde SOLO con el código de las pruebas."

In [116]:
def unittest_prompt_for(code):
    prompt = "Genera pruebas unitarias completas para el siguiente código.\n"
    prompt += "Detecta automáticamente el lenguaje y usa el framework de testing más apropiado.\n"
    prompt += "Incluye: casos normales, edge cases y casos de error.\n\n"
    prompt += "Código:\n" + code
    return prompt

def messages_for_unittest(code):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": unittest_prompt_for(code)}
    ]

In [117]:
def generate_tests_gpt(code):
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for_unittest(code),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```javascript\n','').replace('```java\n','').replace('```','').strip()

def generate_tests_claude(code):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2500,
        system=unittest_system_message,
        messages=[{"role": "user", "content": unittest_prompt_for(code)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```python\n','').replace('```javascript\n','').replace('```java\n','').replace('```','').strip()

def generate_tests(code, model):
    if model == "GPT":
        result = generate_tests_gpt(code)
    elif model == "Claude":
        result = generate_tests_claude(code)
    else:
        raise ValueError("Modelo desconocido")
    
    for stream_so_far in result:
        yield stream_so_far

In [118]:
test_example = """def calcular_promedio(numeros):
    if not numeros:
        raise ValueError("La lista no puede estar vacía")
    return sum(numeros) / len(numeros)

def es_primo(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True"""

with gr.Blocks(css=css) as test_ui:
    gr.Markdown("## Generador Automático de Pruebas Unitarias")
    gr.Markdown("*Detecta automáticamente el lenguaje y genera tests con el framework apropiado*")

    with gr.Row():
        code_input = gr.Code(
            label="Código a Testear:",
            lines=15,
            value=test_example
        )
        test_output = gr.Code(
            label="Pruebas Unitarias Generadas:",
            lines=15
        )

    with gr.Row():
        model = gr.Dropdown(
            ["GPT", "Claude"],
            label="Modelo",
            value="GPT"
        )
        gen_btn = gr.Button(" Generar Tests", variant="primary")

    gen_btn.click(
        generate_tests,
        inputs=[code_input, model],
        outputs=[test_output]
    )

test_ui.launch(inbrowser=True)

/tmp/ipykernel_6982/3684487733.py:14: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=css) as test_ui:


[37124:37124:0100/000000.300107:ERROR:content/zygote/zygote_linux.cc:673] write: Broken pipe (32)


# 5. Crea un generador de código que escribe código comercial para comprar y vender acciones en un entorno simulado, basado en una API determinada